### Import TensorFlow and other libraries

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

### Read the data

First, look in the text:

In [4]:
# Read, then decode for py2 compat.
text = open('bncorpus200h.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))
print('Number of words:', len(text.split(" ")))

Length of text: 10278878 characters
Number of words: 1204459


In [3]:
# Take a look at the first 250 characters in text
print(text[:250])

merhaba saat on üç haberleri ajanstan alıyorsunuz italyan futbolcu materazzi kendisine kafa atan zidanne'a hakaret ettiğini itiraf etti
ancak tam olarak ne söylediğini belirtmedi
imf bir kez daha çalışanları kızdırıcak imf işten çıkarmaların kolaylaş


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

36 unique characters


## Process the text

### Vectorize the text

Before training, you need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [5]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now you have an integer representation for each character. Notice that you mapped the character as indexes from 0 to `len(unique)`.

In [6]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  "'" :   2,
  '2' :   3,
  'a' :   4,
  'b' :   5,
  'c' :   6,
  'd' :   7,
  'e' :   8,
  'f' :   9,
  'g' :  10,
  'h' :  11,
  'i' :  12,
  'j' :  13,
  'k' :  14,
  'l' :  15,
  'm' :  16,
  'n' :  17,
  'o' :  18,
  'p' :  19,
  ...
}


In [7]:
# Show how the first 13 characters from the text are mapped to integers
print('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'merhaba saat ' ---- characters mapped to int ---- > [16  8 21 11  4  5  4  1 22  4  4 23  1]


### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [8]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

m
e
r
h
a


The `batch` method lets us easily convert these individual characters to sequences of the desired size.

In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'merhaba saat on üç haberleri ajanstan alıyorsunuz italyan futbolcu materazzi kendisine kafa atan zida'
"nne'a hakaret ettiğini itiraf etti\nancak tam olarak ne söylediğini belirtmedi\nimf bir kez daha çalışa"
"nları kızdırıcak imf işten çıkarmaların kolaylaştırılmasını istiyor imf uzmanları türkiye'de asgari ü"
'creti de çok yüksek buluyor\nisrail gazze operasyonunu bitirme niyetinde değil israil hükümeti gerekir'
'se\noperasyonu daha da şiddetlendirme kararı aldı\nfilistinlilerin dramıysa her geçen gün artıyor gazze'


For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [11]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'merhaba saat on üç haberleri ajanstan alıyorsunuz italyan futbolcu materazzi kendisine kafa atan zid'
Target data: 'erhaba saat on üç haberleri ajanstan alıyorsunuz italyan futbolcu materazzi kendisine kafa atan zida'


Each index of these vectors is processed as a one time step. For the input at time step 0, the model receives the index for "F" and tries to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [12]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 16 ('m')
  expected output: 8 ('e')
Step    1
  input: 8 ('e')
  expected output: 21 ('r')
Step    2
  input: 21 ('r')
  expected output: 11 ('h')
Step    3
  input: 11 ('h')
  expected output: 4 ('a')
Step    4
  input: 4 ('a')
  expected output: 5 ('b')


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [38]:
# Batch size
BATCH_SIZE = 1

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((1, 256, 100), (1, 256, 100)), types: (tf.int32, tf.int32)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [39]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]))
    model.add(tf.keras.layers.LSTM(rnn_units * 2, return_sequences=True, stateful=True))
    model.add(tf.keras.layers.LSTM(rnn_units * 2, return_sequences=True, stateful=True))
    model.add(tf.keras.layers.LSTM(rnn_units * 2, return_sequences=True, stateful=True))
    model.add(tf.keras.layers.Dense(vocab_size))
    return model

In [16]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

Please note that Keras sequential model is used here since all the layers in the model only have single input and produce single output. In case you want to retrieve and reuse the states from stateful RNN layer, you might want to build your model with Keras functional API or model subclassing. Please check [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse) for more details.

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [40]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            9216      
_________________________________________________________________
lstm_3 (LSTM)                (1, None, 2048)           18882560  
_________________________________________________________________
lstm_4 (LSTM)                (1, None, 2048)           33562624  
_________________________________________________________________
lstm_5 (LSTM)                (1, None, 2048)           33562624  
_________________________________________________________________
dense_1 (Dense)              (1, None, 36)             73764     
Total params: 86,090,788
Trainable params: 86,090,788
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [19]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [20]:
sampled_indices

array([ 1, 35, 23, 33, 25,  1, 18, 14,  6, 34, 33, 27, 26,  8, 11,  0,  0,
       18,  1, 32, 30, 22, 24, 27,  5, 29, 26, 11, 29, 26, 23, 22,  2, 29,
        3, 15, 30, 31,  8, 27,  1, 12,  1, 18, 34, 26, 13, 26, 22, 21, 13,
       30, 27, 26, 15, 34,  9, 15,  0, 26,  7,  0, 16, 28,  9, 34,  9, 34,
       32, 20, 35,  2,  6, 27, 12, 29, 22,  3,  1, 26, 27, 22,  4, 26, 22,
       29,  8, 34, 29, 35,  5,  0, 29, 27,  1, 17,  4, 22, 19, 35],
      dtype=int64)

Decode these to see the text predicted by this untrained model:

In [21]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "'daki maçın önemini hayatımda en çok istediğim şey futbolcu veya antrenör olarak bu sahada bulunmakt"

Next Char Predictions: 
 " ştğv okcığxweh\n\no üçsuxbzwhzwts'z2lçöex i oıwjwsrjçxwlıfl\nwd\nmyfıfıüqş'cxizs2 wxsawszeızşb\nzx naspş"


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [41]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (256, 100, 36)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       3.583403


Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [43]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [25]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [ ]:
EPOCHS = 100
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 2, verbose = 0, restore_best_weights = True)
history = model.fit(dataset, epochs=EPOCHS, callbacks=[early_stopping, checkpoint_callback])

## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, you need to rebuild the model and restore the weights from the checkpoint.


In [45]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_27'

In [46]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [47]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            9216      
_________________________________________________________________
lstm_6 (LSTM)                (1, None, 2048)           18882560  
_________________________________________________________________
lstm_7 (LSTM)                (1, None, 2048)           33562624  
_________________________________________________________________
lstm_8 (LSTM)                (1, None, 2048)           33562624  
_________________________________________________________________
dense_2 (Dense)              (1, None, 36)             73764     
Total params: 86,090,788
Trainable params: 86,090,788
Non-trainable params: 0
_________________________________________________________________


In [242]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 250

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [269]:
print(generate_text(model, start_string= "istanbul'da üç kişi"))

istanbul'da üç kişinin cesedi bulundu
bağdat'ta dün özel bir güvenlik şirketine ait helikopterlerinin sayısı
altmış beş bin artarak
almanya'nın başkenti berlin'de geçen cuma günü sayısı on altıya kadar oylanacak
yasa tasarısının geçmesini önlemeye çalışmasının beklendi


The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.

So now that you've seen how to run the model manually let's unpack the training loop, and implement it ourselves. This gives a starting point if, for example, you want to implement _curriculum learning_ to help stabilize the model's open-loop output.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The procedure works as follows:

* First, reset the RNN state. You do this by calling the `tf.keras.Model.reset_states` method.

* Next, iterate over the dataset (batch by batch) and calculate the *predictions* associated with each.

* Open a `tf.GradientTape`, and calculate the predictions and loss in that context.

* Calculate the gradients of the loss with respect to the model variables using the `tf.GradientTape.grads` method.

* Finally, take a step downwards by using the optimizer's `tf.train.Optimizer.apply_gradients` method.


In [38]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [39]:
optimizer = tf.keras.optimizers.Adam()

In [40]:
@tf.function
def train_step(inp, target):
    with tf.GradientTape() as tape:
        predictions = model(inp)
        loss = tf.reduce_mean(
            tf.keras.losses.sparse_categorical_crossentropy(
                target, predictions, from_logits=True))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

In [ ]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()

    # resetting the hidden state at the start of every epoch
    model.reset_states()

    for (batch_n, (inp, target)) in enumerate(dataset):
        loss = train_step(inp, target)

        if batch_n % 100 == 0:
            template = 'Epoch {} Batch {} Loss {}'
            print(template.format(epoch + 1, batch_n, loss))

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print('Epoch {} Loss {:.4f}'.format(epoch + 1, loss))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))